In [52]:
from fastai.callback.all import Callback, hook_outputs
from fastai.vision.all import (Adam, DataBlock, download_images, get_image_files, 
                               ImageBlock, ImageDataLoaders, IndexSplitter, Normalize, 
                               NormType, parent_label, PILImage, RandomSplitter, Resize, 
                               ResizeMethod, RMSProp, verify_images, unet_learner,
                               xresnet18)
from fastai.vision.gan import GANLearner
import math
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
from pathlib import Path
import PIL
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models.vgg import vgg19
from typing import List, Tuple

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
img_size = 224
n_channels = 3
bs = 32

# Data

## Target ds

In [ ]:
anime_data = pd.read_csv('/kaggle/input/safebooru/all_data.csv')

In [ ]:
anime_data

In [75]:
anime_faces_path = Path('./anime-faces').resolve()
!mkdir $anime_faces_path

In [76]:
anime_faces_rows = anime_data[anime_data['tags'].str.contains('face')]
anime_faces_urls = list(anime_faces_rows['sample_url'].apply(lambda url: 'http:' + url))
len(anime_faces_urls)

102977

In [77]:
download_images(anime_faces_path, urls=anime_faces_urls[:10000])

In [82]:
failed_imgs = verify_images(anime_faces_path.ls())
failed_imgs.map(Path.unlink)
!ls $anime_faces_path | wc -l # should be 9790

9790


In [84]:
# Zip chosen subset of original ds in order to ease download or dataset creation
!zip -r anime_faces_safebooru.zip $anime_faces_path

  adding: kaggle/working/anime-faces/ (stored 0%)
  adding: kaggle/working/anime-faces/00009967.jpg (deflated 2%)
  adding: kaggle/working/anime-faces/00001294.jpg (deflated 0%)
  adding: kaggle/working/anime-faces/00005471.jpg (deflated 0%)
  adding: kaggle/working/anime-faces/00008554.jpg (deflated 2%)
  adding: kaggle/working/anime-faces/00008588.jpg (deflated 2%)
  adding: kaggle/working/anime-faces/00004971.png (deflated 1%)
  adding: kaggle/working/anime-faces/00009672.jpg (deflated 0%)
  adding: kaggle/working/anime-faces/00008895.jpg (deflated 0%)
  adding: kaggle/working/anime-faces/00004679.jpg (deflated 0%)
  adding: kaggle/working/anime-faces/00002641.jpg (deflated 1%)
  adding: kaggle/working/anime-faces/00007201.jpg (deflated 0%)
  adding: kaggle/working/anime-faces/00001175.jpg (deflated 3%)
  adding: kaggle/working/anime-faces/00001132.png (deflated 0%)
  adding: kaggle/working/anime-faces/00006536.jpg (deflated 2%)
  adding: kaggle/working/anime-faces/00003184.jpg (def

  adding: kaggle/working/anime-faces/00002281.jpg (deflated 1%)
  adding: kaggle/working/anime-faces/00002410.jpg (deflated 1%)
  adding: kaggle/working/anime-faces/00009748.jpg (deflated 3%)
  adding: kaggle/working/anime-faces/00005714.jpg (deflated 13%)
  adding: kaggle/working/anime-faces/00000701.jpg (deflated 4%)
  adding: kaggle/working/anime-faces/00009647.jpg (deflated 5%)
  adding: kaggle/working/anime-faces/00009013.jpg (deflated 9%)
  adding: kaggle/working/anime-faces/00003528.jpg (deflated 7%)
  adding: kaggle/working/anime-faces/00000537.jpg (deflated 9%)
  adding: kaggle/working/anime-faces/00003139.jpg (deflated 3%)
  adding: kaggle/working/anime-faces/00002156.jpg (deflated 1%)
  adding: kaggle/working/anime-faces/00007524.jpg (deflated 16%)
  adding: kaggle/working/anime-faces/00007595.jpg (deflated 3%)
  adding: kaggle/working/anime-faces/00002292.jpg (deflated 0%)
  adding: kaggle/working/anime-faces/00003704.jpg (deflated 2%)
  adding: kaggle/working/anime-faces/0

  adding: kaggle/working/anime-faces/00007751.jpg (deflated 1%)
  adding: kaggle/working/anime-faces/00006387.jpg (deflated 15%)
  adding: kaggle/working/anime-faces/00003671.jpg (deflated 4%)
  adding: kaggle/working/anime-faces/00009295.jpg (deflated 0%)
  adding: kaggle/working/anime-faces/00006129.jpg (deflated 8%)
  adding: kaggle/working/anime-faces/00009103.jpg (deflated 3%)
  adding: kaggle/working/anime-faces/00007355.jpg (deflated 1%)
  adding: kaggle/working/anime-faces/00002027.jpg (deflated 5%)
  adding: kaggle/working/anime-faces/00004675.png (deflated 0%)
  adding: kaggle/working/anime-faces/00004815.jpg (deflated 3%)
  adding: kaggle/working/anime-faces/00007882.jpg (deflated 6%)
  adding: kaggle/working/anime-faces/00008977.jpg (deflated 1%)
  adding: kaggle/working/anime-faces/00009718.jpg (deflated 10%)
  adding: kaggle/working/anime-faces/00006962.jpg (deflated 1%)
  adding: kaggle/working/anime-faces/00004650.png (deflated 0%)
  adding: kaggle/working/anime-faces/0

  adding: kaggle/working/anime-faces/00005877.jpg (deflated 8%)
  adding: kaggle/working/anime-faces/00004676.png (deflated 0%)
  adding: kaggle/working/anime-faces/00004412.jpg (deflated 1%)
  adding: kaggle/working/anime-faces/00007395.jpg (deflated 2%)
  adding: kaggle/working/anime-faces/00008049.jpg (deflated 1%)
  adding: kaggle/working/anime-faces/00006759.jpg (deflated 0%)
  adding: kaggle/working/anime-faces/00008645.jpg (deflated 2%)
  adding: kaggle/working/anime-faces/00003720.jpg (deflated 15%)
  adding: kaggle/working/anime-faces/00002087.jpg (deflated 2%)
  adding: kaggle/working/anime-faces/00008579.jpg (deflated 4%)
  adding: kaggle/working/anime-faces/00009288.jpg (deflated 4%)
  adding: kaggle/working/anime-faces/00006878.jpg (deflated 1%)
  adding: kaggle/working/anime-faces/00002370.jpg (deflated 2%)
  adding: kaggle/working/anime-faces/00006836.jpg (deflated 3%)
  adding: kaggle/working/anime-faces/00001665.jpg (deflated 2%)
  adding: kaggle/working/anime-faces/00

  adding: kaggle/working/anime-faces/00007495.jpg (deflated 0%)
  adding: kaggle/working/anime-faces/00003053.jpg (deflated 32%)
  adding: kaggle/working/anime-faces/00007133.jpg (deflated 2%)
  adding: kaggle/working/anime-faces/00009996.jpg (deflated 1%)
  adding: kaggle/working/anime-faces/00002197.jpg (deflated 0%)
  adding: kaggle/working/anime-faces/00005219.jpg (deflated 9%)
  adding: kaggle/working/anime-faces/00003853.jpg (deflated 3%)
  adding: kaggle/working/anime-faces/00007370.jpg (deflated 1%)
  adding: kaggle/working/anime-faces/00009259.jpg (deflated 3%)
  adding: kaggle/working/anime-faces/00000959.jpg (deflated 1%)
  adding: kaggle/working/anime-faces/00006537.jpg (deflated 5%)
  adding: kaggle/working/anime-faces/00001694.jpg (deflated 2%)
  adding: kaggle/working/anime-faces/00002128.jpg (deflated 0%)
  adding: kaggle/working/anime-faces/00005659.jpg (deflated 7%)
  adding: kaggle/working/anime-faces/00006180.png (deflated 0%)
  adding: kaggle/working/anime-faces/00

...

  adding: kaggle/working/anime-faces/00009306.jpg (deflated 2%)
  adding: kaggle/working/anime-faces/00007109.jpg (deflated 1%)
  adding: kaggle/working/anime-faces/00009147.jpg (deflated 11%)
  adding: kaggle/working/anime-faces/00002487.jpg (deflated 1%)
  adding: kaggle/working/anime-faces/00007680.jpg (deflated 1%)
  adding: kaggle/working/anime-faces/00002882.jpg (deflated 5%)
  adding: kaggle/working/anime-faces/00006378.jpg (deflated 0%)
  adding: kaggle/working/anime-faces/00002928.jpg (deflated 6%)
  adding: kaggle/working/anime-faces/00000228.png (deflated 0%)
  adding: kaggle/working/anime-faces/00001530.jpg (deflated 1%)
  adding: kaggle/working/anime-faces/00006051.jpg (deflated 0%)
  adding: kaggle/working/anime-faces/00006042.jpg (deflated 8%)
  adding: kaggle/working/anime-faces/00004108.jpg (deflated 1%)
  adding: kaggle/working/anime-faces/00005177.jpg (deflated 0%)
  adding: kaggle/working/anime-faces/00005942.jpg (deflated 3%)
  adding: kaggle/working/anime-faces/00

  adding: kaggle/working/anime-faces/00001836.jpg (deflated 5%)
  adding: kaggle/working/anime-faces/00000306.jpg (deflated 1%)
  adding: kaggle/working/anime-faces/00009461.jpg (deflated 0%)
  adding: kaggle/working/anime-faces/00007677.jpg (deflated 4%)
  adding: kaggle/working/anime-faces/00009075.jpg (deflated 3%)
  adding: kaggle/working/anime-faces/00005502.jpg (deflated 1%)
  adding: kaggle/working/anime-faces/00000266.jpg (deflated 3%)
  adding: kaggle/working/anime-faces/00001725.jpg (deflated 13%)
  adding: kaggle/working/anime-faces/00000149.png (deflated 0%)
  adding: kaggle/working/anime-faces/00008922.png (deflated 0%)
  adding: kaggle/working/anime-faces/00005283.jpg (deflated 0%)
  adding: kaggle/working/anime-faces/00000617.jpg (deflated 1%)
  adding: kaggle/working/anime-faces/00007318.jpg (deflated 4%)
  adding: kaggle/working/anime-faces/00002329.jpg (deflated 0%)
  adding: kaggle/working/anime-faces/00009569.jpg (deflated 2%)
  adding: kaggle/working/anime-faces/00

  adding: kaggle/working/anime-faces/00009975.jpg (deflated 0%)
  adding: kaggle/working/anime-faces/00005774.png (deflated 0%)
  adding: kaggle/working/anime-faces/00003950.jpg (deflated 5%)
  adding: kaggle/working/anime-faces/00001821.jpg (deflated 4%)
  adding: kaggle/working/anime-faces/00000207.jpg (deflated 0%)
  adding: kaggle/working/anime-faces/00000671.jpg (deflated 0%)
  adding: kaggle/working/anime-faces/00005251.jpg (deflated 4%)
  adding: kaggle/working/anime-faces/00003927.png (deflated 0%)
  adding: kaggle/working/anime-faces/00004280.png (deflated 1%)
  adding: kaggle/working/anime-faces/00003437.jpg (deflated 2%)
  adding: kaggle/working/anime-faces/00001372.jpg (deflated 0%)
  adding: kaggle/working/anime-faces/00000848.jpg (deflated 9%)
  adding: kaggle/working/anime-faces/00005760.jpg (deflated 11%)
  adding: kaggle/working/anime-faces/00001811.jpg (deflated 9%)
  adding: kaggle/working/anime-faces/00001732.jpg (deflated 1%)
  adding: kaggle/working/anime-faces/00

  adding: kaggle/working/anime-faces/00001723.jpg (deflated 1%)
  adding: kaggle/working/anime-faces/00005923.jpg (deflated 4%)
  adding: kaggle/working/anime-faces/00006806.jpg (deflated 2%)
  adding: kaggle/working/anime-faces/00004279.jpg (deflated 4%)
  adding: kaggle/working/anime-faces/00005944.jpg (deflated 1%)
  adding: kaggle/working/anime-faces/00005725.jpg (deflated 7%)
  adding: kaggle/working/anime-faces/00001820.jpg (deflated 1%)
  adding: kaggle/working/anime-faces/00008637.png (deflated 0%)
  adding: kaggle/working/anime-faces/00001526.jpg (deflated 1%)
  adding: kaggle/working/anime-faces/00004306.jpg (deflated 10%)
  adding: kaggle/working/anime-faces/00001224.jpg (deflated 0%)
  adding: kaggle/working/anime-faces/00002971.jpg (deflated 2%)
  adding: kaggle/working/anime-faces/00000376.jpg (deflated 0%)
  adding: kaggle/working/anime-faces/00000734.jpg (deflated 0%)
  adding: kaggle/working/anime-faces/00001282.jpg (deflated 1%)
  adding: kaggle/working/anime-faces/00

  adding: kaggle/working/anime-faces/00006747.jpg (deflated 0%)
  adding: kaggle/working/anime-faces/00005189.jpg (deflated 0%)
  adding: kaggle/working/anime-faces/00004786.jpg (deflated 2%)
  adding: kaggle/working/anime-faces/00008043.jpg (deflated 2%)
  adding: kaggle/working/anime-faces/00004551.jpg (deflated 8%)
  adding: kaggle/working/anime-faces/00001416.jpg (deflated 1%)
  adding: kaggle/working/anime-faces/00008426.jpg (deflated 3%)
  adding: kaggle/working/anime-faces/00003823.jpg (deflated 5%)
  adding: kaggle/working/anime-faces/00004000.jpg (deflated 3%)
  adding: kaggle/working/anime-faces/00008394.jpg (deflated 4%)
  adding: kaggle/working/anime-faces/00000517.jpg (deflated 3%)
  adding: kaggle/working/anime-faces/00004742.jpg (deflated 3%)
  adding: kaggle/working/anime-faces/00000996.png (deflated 0%)
  adding: kaggle/working/anime-faces/00002002.jpg (deflated 2%)
  adding: kaggle/working/anime-faces/00000405.jpg (deflated 1%)
  adding: kaggle/working/anime-faces/000

## Input ds

In [ ]:
celeba_path = Path('/kaggle/input/celeba-dataset/img_align_celeba/img_align_celeba')
#input_fns = get_image_files(celeba_path)
# get_image_files is too slow, there's no need to check the extension here
input_fns = celeba_path.ls()
input_fns

The ds path passed to `dblock.dataloaders()` or `ImageDataLoaders.from_dblock()` will be forwarded
to `get_items`, which will return a list of items, usually a list of image paths if `get_items=get_image_files`.

So, for each item, we are expected to receive a filename `fn` and be able to
derive x and y from it, with `get_x(fn)` and `get_y(fn)`.

For unpaired image to image translation, we can:
* Use the target images ds path as the DataBlock `source`. Then, `get_y` can just return the path received.
* Load independently the filenames of the input images ds; let's call it `input_fns`. Then, `get_x` would need to
  return a random item from `input_fns`.
  
The problem here could be that we are tying each input image to the same target image. However, if batches are shuffled, it just means that they will always be part of the same (x, y) batch; I guess it may be worse for low batch sizes.

---

In [ ]:
def get_random_input(fn):
    return input_fns[random.randint(0, len(input_fns)-1)]
    
dblock = DataBlock(blocks = (ImageBlock, ImageBlock),
                   get_x = get_random_input,
                   get_items = get_image_files,
                   splitter = IndexSplitter([]),
                   item_tfms=Resize(img_size, method=ResizeMethod.Crop), 
                   batch_tfms = Normalize.from_stats(torch.tensor([0.5,0.5,0.5]), torch.tensor([0.5,0.5,0.5])))
main_path = anime_faces_path
dls = dblock.dataloaders(main_path, path=main_path, bs=bs)

In [ ]:
dls.show_batch()

# Loss function

## Content loss

In [ ]:
class FeaturesCalculator:
    def __init__(self, vgg_style_layers_idx:List[int], vgg_content_layers_idx:List[int],
                 vgg:nn.Module=None, normalize_inputs=False, device:torch.device=None):
        self.vgg = vgg19(pretrained=True) if vgg is None else vgg
        self.vgg.eval()
        if device is not None: self.vgg.to(device)
        modules_to_hook = [self.vgg.features[idx] for idx in (*vgg_style_layers_idx, *vgg_content_layers_idx)]
        self.hooks = hook_outputs(modules_to_hook, detach=False)
        self.style_ftrs_hooks = self.hooks[:len(vgg_style_layers_idx)]
        self.content_ftrs_hooks = self.hooks[len(vgg_style_layers_idx):]
        self.normalize_inputs = normalize_inputs
        # TODO: when to remove hooks???
        #  `clean` method????
    
    def _get_hooks_out(self, hooks):
        return [h.stored for h in hooks]
    
    def _forward(img_t:torch.Tensor):
        if self.normalize_inputs: 
            mean, std = fastai.vision.imagenet_stats
            img_t = fastai.vision.normalize(img_t, torch.tensor(mean), torch.tensor(std))
        self.vgg(img_t)
    
    def calc_style(self, img_t:torch.Tensor) -> List[torch.Tensor]:
        self.vgg(img_t)
        return self._get_hooks_out(self.style_ftrs_hooks)
    
    def calc_content(self, img_t:torch.Tensor) -> List[torch.Tensor]:
        self.vgg(img_t)
        return self._get_hooks_out(self.content_ftrs_hooks)
    
    def calc_style_and_content(self, img_t:torch.Tensor) -> Tuple[List[torch.Tensor], List[torch.Tensor]]:
        self.vgg(img_t)
        style_ftrs = self._get_hooks_out(self.style_ftrs_hooks)
        content_ftrs = self._get_hooks_out(self.content_ftrs_hooks)
        return style_ftrs, content_ftrs

In [ ]:
vgg_content_layers_idx = [22]
ftrs_calc = FeaturesCalculator([], vgg_content_layers_idx, device=device)

In the next cell, there are two versions of content loss, with the same behaviour but different implementation:
* A functional version, returned by `get_content_loss`. A callback parameter is needed 
* A callback version. Although less intuitive, this may be preferable because it stores the loss value inside `learner.loss_func.content_loss`, making it accessible to the metrics display system. If you want the learner to display the content loss every epoch, it only requires passing `metrics=['content_loss', ...]` to GANLearner.init.

In [ ]:
content_loss_func = nn.MSELoss(reduction='mean')


def get_content_loss(last_input_cb):
    def _content_loss(output, target):
        input_content_ftrs = ftrs_calc.calc_content(last_input_cb.x)[0]
        output_content_ftrs = ftrs_calc.calc_content(output)[0]
        return content_loss_func(output_content_ftrs, input_content_ftrs)
    
    return _content_loss


class ContentLossCallback(Callback):
    def __init__(self, weight=1., ftrs_calc=None, device=None):
        self.weight = weight
        if ftrs_calc is None:
            vgg_content_layers_idx = [22]
            ftrs_calc = FeaturesCalculator([], vgg_content_layers_idx, device=device)
        self.ftrs_calc = ftrs_calc
        
    def after_loss(self):
        if self.gan_trainer.gen_mode:
            input_content_ftrs = self.ftrs_calc.calc_content(self.x)[0]
            output_content_ftrs = self.ftrs_calc.calc_content(self.pred)[0]
            loss_val = content_loss_func(output_content_ftrs, input_content_ftrs)
            # Store result inside learn.loss_func to make it visible to metrics display
            self.learn.loss_func.content_loss = loss_val            
            # This will probably stop working once a new fastai version is released, as backward 
            # won't be called on learn.loss anymore, but on learn.loss_grad
            self.learn.loss += loss_val

The following functions are only useful if you decide to use the functional version of content loss and want to combine it with Wasserstein loss:

In [ ]:
def gen_wgan_loss(fake_pred, output, target): return fake_pred.mean()
def crit_wgan_loss(real_pred, fake_pred): return real_pred.mean() - fake_pred.mean()


def get_gen_wgan_content_loss(last_input_cb, content_loss_w=1.):
    content_loss = get_content_loss(last_input_cb)
    
    def _gen_wgan_content_loss(fake_pred, output, target):
        wgan_loss = gen_wgan_loss(fake_pred, output, target) 
        cont_loss = content_loss(output, target)
        return wgan_loss + content_loss_w * cont_loss
    
    return _gen_wgan_content_loss


def create_wgan_w_content_loss_learner(dls, generator, critic, cbs=None, **kwargs):
    if cbs is None: cbs = []
    last_input_cb=Callback()
    cbs.append(last_input_cb)
    gen_loss = get_gen_wgan_content_loss(last_input_cb)
    return GANLearner(dls, generator, critic, gen_loss, crit_wgan_loss,
                      cbs=cbs, **kwargs)

## Gradient penalty

In [ ]:
class GANGPCallback(Callback):
    def __init__(self, plambda=10., epsilon_sampler=None): 
        self.plambda = plambda
        if epsilon_sampler is None: epsilon_sampler = random_epsilon_gp_sampler
        self.epsilon_sampler = epsilon_sampler
        
    def _gradient_penalty(self, real, fake, plambda, epsilon_sampler):
        epsilon = epsilon_sampler(real, fake)
        x_hat = epsilon * real + (1 - epsilon) * fake
        x_hat_pred = self.model.critic(x_hat).mean()

        grads = torch.autograd.grad(outputs=x_hat_pred, inputs=x_hat, create_graph=True)[0]
        return plambda * ((grads.norm() - 1)**2)    
        
    def after_loss(self):
        if not self.gan_trainer.gen_mode:
            # In critic mode, GANTrainer swaps x and y; so, here x is original y (real target)
            real = self.x
            assert not self.y.requires_grad
            fake = self.model.generator(self.y).requires_grad_(True)
            # This will probably stop working once a new fastai version is released, as backward 
            # won't be called on learn.loss anymore, but on learn.loss_grad
            self.learn.loss += self._gradient_penalty(real, fake, self.plambda, self.epsilon_sampler)


def random_epsilon_gp_sampler(real: torch.Tensor, fake: torch.Tensor) -> torch.Tensor:
    # A different random value of epsilon for any element of a batch
    epsilon_vec = torch.rand(real.shape[0], 1, 1, 1, dtype=torch.float, device=real.device, requires_grad=False)
    return epsilon_vec.expand_as(real)

------------------

# TRAINING

In [ ]:
generator_learner = unet_learner(dls, xresnet18, normalize=True, n_out=n_channels, pretrained=False, 
                                 loss_func=lambda *args: 0, #cbs=[last_input_cb],
                                )
generator = generator_learner.model
critic = xresnet18(n_out=1)
learn = GANLearner.wgan(dls, generator, critic, opt_func = RMSProp)
learn.recorder.train_metrics=True
learn.recorder.valid_metrics=False
learn.fit(1, 2e-4, wd=0.)

In [ ]:
learn.fit(4, 2e-4, wd=0.)

In [ ]:
learn.show_results(ds_idx=0)

In [ ]:
learn.fit(4, 2e-4, wd=0.)

In [ ]:
learn.show_results(ds_idx=0)

In [ ]:
# GENERATOR:
#   -Option A) UNET
#   -Option B) Flat net with only 3x3 stride 1 convolutions
#   -Option C) Accordion
#
#   Input: CelebA images (instead of noise)


# CRITIC:
#   -Option A) XResnet18
#   -Option B) XResnet18 with spectral norm (check if option C isn't exactly this)
#   -Option C) XSE_Resnet?????
#   -Option D) XSE_Resnext
#
#   Output: probably just 1 unit is ideal
#
#   Input:
#       -Real: AnimeDS images
#       -Fake: generator output


# LOSS: 
#   -Initially: WGAN (with weight clipping)
#   -Later: add GP (clipping should be removed)
#   -Finally: add content loss


# Modifications:
# -LeakyReLU for critic
# -Initializations????
#     -Check if, before training, generator output is identical to input

# TODO

* Once decent results appear, compare them with FID.
* Test saving and loading
* UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
* Don't forget get_image_files can make training slower at the beginning
* Add spectral norm
* Add content loss
* Maybe, add cycle consistency loss (this would slow down training), another generator, ... (cycle-GAN)
* If no results are gotten:
  * use bigger dataset
  * use more specific anime faces dataset (it could be worse for production)
* Add/vary transforms
  